# Build a Chatbot


:::info Prerequisites

This guide assumes familiarity with the following concepts:

- [Chat Models](/docs/concepts/chat_models)
- [Prompt Templates](/docs/concepts/prompt_templates)
- [Chat History](/docs/concepts/chat_history)

This guide requires `langgraph >= 0.2.28`.

:::


```{=mdx}

:::note

This tutorial previously built a chatbot using [RunnableWithMessageHistory](https://api.js.langchain.com/classes/_langchain_core.runnables.RunnableWithMessageHistory.html). You can access this version of the tutorial in the [v0.2 docs](https://js.langchain.com/v0.2/docs/tutorials/chatbot/).

The LangGraph implementation offers a number of advantages over `RunnableWithMessageHistory`, including the ability to persist arbitrary components of an application's state (instead of only messages).

:::

```

## Overview

We'll go over an example of how to design and implement an LLM-powered chatbot. 
This chatbot will be able to have a conversation and remember previous interactions.


Note that this chatbot that we build will only use the language model to have a conversation.
There are several other related concepts that you may be looking for:

- [Conversational RAG](/docs/tutorials/qa_chat_history): Enable a chatbot experience over an external source of data
- [Agents](https://langchain-ai.github.io/langgraphjs/tutorials/multi_agent/agent_supervisor/): Build a chatbot that can take actions

This tutorial will cover the basics which will be helpful for those two more advanced topics, but feel free to skip directly to there should you choose.

## Setup

### Jupyter Notebook

This guide (and most of the other guides in the documentation) uses [Jupyter notebooks](https://jupyter.org/) and assumes the reader is as well. Jupyter notebooks are perfect for learning how to work with LLM systems because oftentimes things can go wrong (unexpected output, API down, etc) and going through guides in an interactive environment is a great way to better understand them.

This and other tutorials are perhaps most conveniently run in a Jupyter notebook. See [here](https://jupyter.org/install) for instructions on how to install.

### Installation

For this tutorial we will need `@langchain/core` and `langgraph`:

```{=mdx}
import Npm2Yarn from "@theme/Npm2Yarn"

<Npm2Yarn>
  @langchain/core @langchain/langgraph uuid
</Npm2Yarn>
```

For more details, see our [Installation guide](/docs/how_to/installation).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```typescript
process.env.LANGCHAIN_TRACING_V2 = "true"
process.env.LANGCHAIN_API_KEY = "..."
```

## Quickstart

First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangeably - select the one you want to use below!

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [27]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({ model: "gpt-4o-mini" })

Let's first use the model directly. `ChatModel`s are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [28]:
await llm.invoke([{ role: "user", content: "Hi im bob" }])

AIMessage {
  "id": "chatcmpl-ABUXeSO4JQpxO96lj7iudUptJ6nfW",
  "content": "Hi Bob! How can I assist you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 10,
      "promptTokens": 10,
      "totalTokens": 20
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 10,
    "output_tokens": 10,
    "total_tokens": 20
  }
}


The model on its own does not have any concept of state. For example, if you ask a followup question:

In [29]:
await llm.invoke([{ role: "user", content: "Whats my name" }])

AIMessage {
  "id": "chatcmpl-ABUXe1Zih4gMe3XgotWL83xeWub2h",
  "content": "I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me during our conversation. If you'd like to tell me your name, feel free to do so!",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 39,
      "promptTokens": 10,
      "totalTokens": 49
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 10,
    "output_tokens": 39,
    "total_tokens": 49
  }
}


Let's take a look at the example [LangSmith trace](https://smith.langchain.com/public/3b768e44-a319-453a-bd6e-30f9df75f16a/r)

We can see that it doesn't take the previous conversation turn into context, and cannot answer the question.
This makes for a terrible chatbot experience!

To get around this, we need to pass the entire conversation history into the model. Let's see what happens when we do that:

In [30]:
await llm.invoke([
  { role: "user", content: "Hi! I'm Bob" },
  { role: "assistant", content: "Hello Bob! How can I assist you today?" },
  { role: "user", content: "What's my name?" }
]);

AIMessage {
  "id": "chatcmpl-ABUXfX4Fnp247rOxyPlBUYMQgahj2",
  "content": "Your name is Bob! How can I help you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 12,
      "promptTokens": 33,
      "totalTokens": 45
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 33,
    "output_tokens": 12,
    "total_tokens": 45
  }
}


And now we can see that we get a good response!

This is the basic idea underpinning a chatbot's ability to interact conversationally.
So how do we best implement this?

## Message persistence

[LangGraph](https://langchain-ai.github.io/langgraphjs/) implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.

Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.

LangGraph comes with a simple in-memory checkpointer, which we use below.

In [31]:
import { START, END, MessagesAnnotation, StateGraph, MemorySaver } from "@langchain/langgraph";

// Define the function that calls the model
const callModel = async (state: typeof MessagesAnnotation.State) => {
  const response = await llm.invoke(state.messages);
  return { messages: response };
};

// Define a new graph
const workflow = new StateGraph(MessagesAnnotation)
  // Define the node and edge
  .addNode("model", callModel)
  .addEdge(START, "model")
  .addEdge("model", END);

// Add memory
const memory = new MemorySaver();
const app = workflow.compile({ checkpointer: memory });

We now need to create a `config` that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a `thread_id`. This should look like:

In [32]:
import { v4 as uuidv4 } from "uuid";

const config = { configurable: { thread_id: uuidv4() } };

This enables us to support multiple conversation threads with a single application, a common requirement when your application has multiple users.

We can then invoke the application:

In [33]:
const input = [
  {
    role: "user",
    content: "Hi! I'm Bob.",
  }
]
const output = await app.invoke({ messages: input }, config)
// The output contains all messages in the state.
// This will long the last message in the conversation.
console.log(output.messages[output.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABUXfjqCno78CGXCHoAgamqXG1pnZ",
  "content": "Hi Bob! How can I assist you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 10,
      "promptTokens": 12,
      "totalTokens": 22
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 12,
    "output_tokens": 10,
    "total_tokens": 22
  }
}


In [34]:
const input2 = [
  {
    role: "user",
    content: "What's my name?",
  }
]
const output2 = await app.invoke({ messages: input2 }, config)
console.log(output2.messages[output2.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABUXgzHFHk4KsaNmDJyvflHq4JY2L",
  "content": "Your name is Bob! How can I help you today, Bob?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 14,
      "promptTokens": 34,
      "totalTokens": 48
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 34,
    "output_tokens": 14,
    "total_tokens": 48
  }
}


Great! Our chatbot now remembers things about us. If we change the config to reference a different `thread_id`, we can see that it starts the conversation fresh.

In [35]:
const config2 = { configurable: { thread_id: uuidv4() } }
const input3 = [
  {
    role: "user",
    content: "What's my name?",
  }
]
const output3 = await app.invoke({ messages: input3 }, config2)
console.log(output3.messages[output3.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABUXhT4EVx8mGgmKXJ1s132qEluxR",
  "content": "I'm sorry, but I don’t have access to personal data about individuals unless it has been shared in the course of our conversation. Therefore, I don't know your name. How can I assist you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 41,
      "promptTokens": 11,
      "totalTokens": 52
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 11,
    "output_tokens": 41,
    "total_tokens": 52
  }
}


However, we can always go back to the original conversation (since we are persisting it in a database)

In [36]:
const output4 = await app.invoke({ messages: input2 }, config)
console.log(output4.messages[output4.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABUXhZmtzvV3kqKig47xxhKEnvVfH",
  "content": "Your name is Bob! If there's anything else you'd like to talk about or ask, feel free!",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 20,
      "promptTokens": 60,
      "totalTokens": 80
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 60,
    "output_tokens": 20,
    "total_tokens": 80
  }
}


This is how we can support a chatbot having conversations with many users!

Right now, all we've done is add a simple persistence layer around the model. We can start to make the more complicated and personalized by adding in a prompt template.

## Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

To add in a system message, we will create a `ChatPromptTemplate`. We will utilize `MessagesPlaceholder` to pass all the messages in.

In [37]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You talk like a pirate. Answer all questions to the best of your ability."],
  new MessagesPlaceholder("messages"),
]);

We can now update our application to incorporate this template:

In [38]:
import { START, END, MessagesAnnotation, StateGraph, MemorySaver } from "@langchain/langgraph";

// Define the function that calls the model
const callModel2 = async (state: typeof MessagesAnnotation.State) => {
  // highlight-start
  const chain = prompt.pipe(llm);
  const response = await chain.invoke(state);
  // highlight-end
  // Update message history with response:
  return { messages: [response] };
};

// Define a new graph
const workflow2 = new StateGraph(MessagesAnnotation)
  // Define the (single) node in the graph
  .addNode("model", callModel2)
  .addEdge(START, "model")
  .addEdge("model", END);

// Add memory
const app2 = workflow2.compile({ checkpointer: new MemorySaver() });

We invoke the application in the same way:

In [39]:
const config3 = { configurable: { thread_id: uuidv4() } }
const input4 = [
  {
    role: "user",
    content: "Hi! I'm Jim.",
  }
]
const output5 = await app2.invoke({ messages: input4 }, config3)
console.log(output5.messages[output5.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABUXio2Vy1YNRDiFdKKEyN3Yw1B9I",
  "content": "Ahoy, Jim! What brings ye to these treacherous waters today? Speak up, matey!",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 22,
      "promptTokens": 32,
      "totalTokens": 54
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 32,
    "output_tokens": 22,
    "total_tokens": 54
  }
}


In [40]:
const input5 = [
  {
    role: "user",
    content:  "What is my name?"
  }
]
const output6 = await app2.invoke({ messages: input5 }, config3)
console.log(output6.messages[output6.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABUXjZNHiT5g7eTf52auWGXDUUcDs",
  "content": "Ye be callin' yerself Jim, if me memory serves me right! Arrr, what else can I do fer ye, matey?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 31,
      "promptTokens": 67,
      "totalTokens": 98
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_3a215618e8"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 67,
    "output_tokens": 31,
    "total_tokens": 98
  }
}


Awesome! Let's now make our prompt a little bit more complicated. Let's assume that the prompt template now looks something like this:

In [41]:
const prompt2 = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."],
  new MessagesPlaceholder("messages"),
]);

Note that we have added a new `language` input to the prompt. Our application now has two parameters-- the input `messages` and `language`. We should update our application's state to reflect this:

In [42]:
import { START, END, StateGraph, MemorySaver, MessagesAnnotation, Annotation } from "@langchain/langgraph";

// Define the State
const GraphAnnotation = Annotation.Root({
  ...MessagesAnnotation.spec,
  language: Annotation<string>(),
});

// Define the function that calls the model
const callModel3 = async (state: typeof GraphAnnotation.State) => {
  const chain = prompt2.pipe(llm);
  const response = await chain.invoke(state);
  return { messages: [response] };
};

const workflow3 = new StateGraph(GraphAnnotation)
  .addNode("model", callModel3)
  .addEdge(START, "model")
  .addEdge("model", END);

const app3 = workflow3.compile({ checkpointer: new MemorySaver() });

In [43]:
const config4 = { configurable: { thread_id: uuidv4() } }
const input6 = {
  messages: [
    {
      role: "user",
      content:  "Hi im bob"
    }
  ],
  language: "Spanish"
}
const output7 = await app3.invoke(input6, config4)
console.log(output7.messages[output7.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABUXkq2ZV9xmOBSM2iJbYSn8Epvqa",
  "content": "¡Hola, Bob! ¿En qué puedo ayudarte hoy?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 12,
      "promptTokens": 32,
      "totalTokens": 44
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 32,
    "output_tokens": 12,
    "total_tokens": 44
  }
}


Note that the entire state is persisted, so we can omit parameters like `language` if no changes are desired:

In [44]:
const input7 = {
  messages: [
    {
      role: "user",
      content:  "What is my name?"
    }
  ],
}
const output8 = await app3.invoke(input7, config4)
console.log(output8.messages[output8.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABUXk9Ccr1dhmA9lZ1VmZ998PFyJF",
  "content": "Tu nombre es Bob. ¿Hay algo más en lo que te pueda ayudar?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 16,
      "promptTokens": 57,
      "totalTokens": 73
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 57,
    "output_tokens": 16,
    "total_tokens": 73
  }
}


To help you understand what's happening internally, check out [this LangSmith trace](https://smith.langchain.com/public/d61630b7-6a52-4dc9-974c-8452008c498a/r).

## Managing Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

**Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.**

We can do this by adding a simple step in front of the prompt that modifies the `messages` key appropriately, and then wrap that new chain in the Message History class. 

LangChain comes with a few built-in helpers for [managing a list of messages](/docs/how_to/#messages). In this case we'll use the [trimMessages](/docs/how_to/trim_messages/) helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages:

In [54]:
import { SystemMessage, HumanMessage, AIMessage, trimMessages } from "@langchain/core/messages"

const trimmer = trimMessages({
  maxTokens: 10,
  strategy: "last",
  tokenCounter: (msgs) => msgs.length,
  includeSystem: true,
  allowPartial: false,
  startOn: "human",
})

const messages = [
    new SystemMessage("you're a good assistant"),
    new HumanMessage("hi! I'm bob"),
    new AIMessage("hi!"),
    new HumanMessage("I like vanilla ice cream"),
    new AIMessage("nice"),
    new HumanMessage("whats 2 + 2"),
    new AIMessage("4"),
    new HumanMessage("thanks"),
    new AIMessage("no problem!"),
    new HumanMessage("having fun?"),
    new AIMessage("yes!"),
]

await trimmer.invoke(messages)

[
  SystemMessage {
    "content": "you're a good assistant",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  HumanMessage {
    "content": "I like vanilla ice cream",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "content": "nice",
    "additional_kwargs": {},
    "response_metadata": {},
    "tool_calls": [],
    "invalid_tool_calls": []
  },
  HumanMessage {
    "content": "whats 2 + 2",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "content": "4",
    "additional_kwargs": {},
    "response_metadata": {},
    "tool_calls": [],
    "invalid_tool_calls": []
  },
  HumanMessage {
    "content": "thanks",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "content": "no problem!",
    "additional_kwargs": {},
    "response_metadata": {},
    "tool_calls": [],
    "invalid_tool_calls": []
  },
  HumanMessage {
    "content": "having fun?",
    "additional_kwargs": {},
    "res

To  use it in our chain, we just need to run the trimmer before we pass the `messages` input to our prompt. 

In [55]:
const callModel4 = async (state: typeof GraphAnnotation.State) => {
  const chain = prompt2.pipe(llm);
  // highlight-start
  const trimmedMessage = await trimmer.invoke(state.messages);
  const response = await chain.invoke({ messages: trimmedMessage, language: state.language });
  // highlight-end
  return { messages: [response] };
};


const workflow4 = new StateGraph(GraphAnnotation)
  .addNode("model", callModel4)
  .addEdge(START, "model")
  .addEdge("model", END);

const app4 = workflow4.compile({ checkpointer: new MemorySaver() });

Now if we try asking the model our name, it won't know it since we trimmed that part of the chat history:

In [56]:
const config5 = { configurable: { thread_id: uuidv4() }}
const input8 = {
  // highlight-next-line
  messages: [...messages, new HumanMessage("What is my name?")],
  language: "English"
}

const output9 = await app4.invoke(
  input8,
  config5,
)
console.log(output9.messages[output9.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABUdCOvzRAvgoxd2sf93oGKQfA9vh",
  "content": "I don’t know your name, but I’d be happy to learn it if you’d like to share!",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 22,
      "promptTokens": 97,
      "totalTokens": 119
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 97,
    "output_tokens": 22,
    "total_tokens": 119
  }
}


But if we ask about information that is within the last few messages, it remembers:

In [57]:
const config6 = { configurable: { thread_id: uuidv4() }}
const input9 = {
  // highlight-next-line
  messages: [...messages, new HumanMessage("What math problem did I ask?")],
  language: "English"
}

const output10 = await app4.invoke(
  input9,
  config6,
)
console.log(output10.messages[output10.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-ABUdChq5JOMhcFA1dB7PvCHLyliwM",
  "content": "You asked for the solution to the math problem \"what's 2 + 2,\" and I answered that it equals 4.",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "completionTokens": 27,
      "promptTokens": 99,
      "totalTokens": 126
    },
    "finish_reason": "stop",
    "system_fingerprint": "fp_1bb46167f9"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 99,
    "output_tokens": 27,
    "total_tokens": 126
  }
}


If you take a look at LangSmith, you can see exactly what is happening under the hood in the [LangSmith trace](https://smith.langchain.com/public/bf1b1a10-0fe0-42f6-9f0f-b70d9f7520dc/r).

## Next Steps

Now that you understand the basics of how to create a chatbot in LangChain, some more advanced tutorials you may be interested in are:

- [Conversational RAG](/docs/tutorials/qa_chat_history): Enable a chatbot experience over an external source of data
- [Agents](https://langchain-ai.github.io/langgraphjs/tutorials/multi_agent/agent_supervisor/): Build a chatbot that can take actions

If you want to dive deeper on specifics, some things worth checking out are:

- [Streaming](/docs/how_to/streaming): streaming is *crucial* for chat applications
- [How to add message history](/docs/how_to/message_history): for a deeper dive into all things related to message history
- [How to manage large message history](/docs/how_to/trim_messages/): more techniques for managing a large chat history
- [LangGraph main docs](https://langchain-ai.github.io/langgraphjs/): for more detail on building with LangGraph